In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_column', None)
df_engagement = pd.read_csv('../data/engagement_clusters.csv', na_values=['?', None, 'undefined'])
df_engagement.sample(5)

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,cluster
3832,33610815039,1,96780.0,379703402.0,1
32740,33658822528,1,188543.0,879620141.0,1
50472,33662929776,1,94570.0,539817659.0,1
7267,33616298389,1,109589.0,331484119.0,1
80408,33684637320,1,12042.0,478120951.0,1


In [3]:
df_experience = pd.read_csv('../data/experience_clusters.csv', na_values=['?', None, 'undefined'])
df_experience.sample(5)

,MSISDN/Number,Total RTT,Total TCP retransmission,Total Throughput,Handset Type,encoded,cluster
33797,33762060392,581.0,19104416.0,245210.0,Huawei B528S-23A,232,1
3696,33626128993,97.0,11790726.0,42785.0,Samsung Galaxy J5 (Sm-J500X),659,0
17776,33663258895,185.0,295832.0,21697.0,Apple iPhone 6 (A1586),41,1
15941,33662061102,548.0,9531120.0,115994.0,Huawei B528S-23A,232,1
1035,33610860102,63.0,322275.0,27627.0,Samsung Galaxy A5 Sm-A520F,612,0


In [4]:
# how many missing values exist or better still what is the % of missing values in the dataset?
def percent_missing(df):

    # Calculate total number of cells in dataframe
    totalCells = np.product(df.shape)

    # Count number of missing values per column
    missingCount = df.isnull().sum()

    # Calculate total number of missing values
    totalMissing = missingCount.sum()

    # Calculate percentage of missing values
    print("The dataset contains", round(((totalMissing/totalCells) * 100), 2), "%", "missing values.")

percent_missing(df_engagement)
percent_missing(df_experience)

The dataset contains 0.0 % missing values.
The dataset contains 0.0 % missing values.


In [5]:
less_engaged_centroid = df_engagement[df_engagement['cluster'] == 0].groupby('cluster').mean()
less_engaged_centroid

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data
cluster,,,,
0,3.367435e+10,2.182889,215418.810195,1.116296e+09


In [6]:
def compute_engagement_score(df,centroid):
    x = float(centroid['Bearer Id'])
    y = float(centroid['Dur. (ms)'])
    z = float(centroid['Total Data'])
    df['engagement score'] = ((df['Bearer Id'] - x)**2 + (df['Dur. (ms)'] - y)**2 + (df['Total Data'] - z)**2)**0.5
    return df
df_engagement = compute_engagement_score(df_engagement,less_engaged_centroid)

In [7]:
df_engagement.head()

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,cluster,engagement score
0,3197020876596,1,877385.0,232123971.0,1,8.841722e+08
1,33601001722,1,116720.0,878690574.0,1,2.376053e+08
2,33601001754,1,181230.0,156859643.0,1,9.594363e+08
3,33601002511,1,134969.0,595966483.0,1,5.203294e+08
4,33601007832,1,49878.0,422320698.0,1,6.939752e+08


In [8]:
worst_experience_centroid = df_experience[df_experience['cluster'] == 0]\
    [['Total RTT', 'Total TCP retransmission', 'Total Throughput', 'encoded', 'cluster']].groupby('cluster').mean()
worst_experience_centroid.head()

,Total RTT,Total TCP retransmission,Total Throughput,encoded
cluster,,,,
0,245.546023,4.195681e+06,32562.62303,706.700761


In [9]:
def compute_experience_score(df,centroid):
    x = float(centroid['Total RTT'])
    y = float(centroid['Total TCP retransmission'])
    z = float(centroid['Total Throughput'])
    w = float(centroid['encoded'])
    df['experience score'] = ((df['Total RTT'] - x)**2 + (df['Total TCP retransmission'] - y)**2 \
                              + (df['Total Throughput'] - z)**2 + (df['encoded'] - w)**2)**0.5
    return df
df_experience = compute_experience_score(df_experience,worst_experience_centroid)
df_experience.head()

,MSISDN/Number,Total RTT,Total TCP retransmission,Total Throughput,Handset Type,encoded,cluster,experience score
0,33601008617,91.0,9370832.0,56781.0,Apple iPhone Se (A1723),62,1,5.175208e+06
1,33601011634,39.0,110232.0,42416.0,Huawei Mate 10 Pro Porsche Design Huawei Mate 10,301,1,4.085460e+06
2,33601021217,160.0,14594645.0,19256.0,Apple iPhone 7 Plus (A1784),54,1,1.039897e+07
3,33601031129,60.0,2325497.0,38190.0,Apple iPhone 8 Plus (A1897),60,1,1.870192e+06
4,33601034530,656.0,2006261.0,8539.0,Apple iPhone 7 (A1778),51,1,2.189551e+06


In [10]:
df_satisfaction = pd.merge(df_engagement[['MSISDN/Number', 'engagement score']], df_experience[['MSISDN/Number', 'experience score']], on="MSISDN/Number")
df_satisfaction.head()

,MSISDN/Number,engagement score,experience score
0,33601008617,3.411151e+08,5.175208e+06
1,33601011634,4.615728e+08,4.085460e+06
2,33601021217,4.862035e+08,1.039897e+07
3,33601031129,4.792428e+08,1.870192e+06
4,33601034530,5.164648e+08,2.189551e+06


In [11]:
df_satisfaction['Satisfaction Score'] = (df_satisfaction['engagement score'] + df_satisfaction['experience score'])/2

In [12]:
df_satisfaction.head()

,MSISDN/Number,engagement score,experience score,Satisfaction Score
0,33601008617,3.411151e+08,5.175208e+06,1.731452e+08
1,33601011634,4.615728e+08,4.085460e+06,2.328292e+08
2,33601021217,4.862035e+08,1.039897e+07,2.483012e+08
3,33601031129,4.792428e+08,1.870192e+06,2.405565e+08
4,33601034530,5.164648e+08,2.189551e+06,2.593272e+08


### Top 10 satisfied customers

In [13]:
df_satisfaction[['MSISDN/Number', 'Satisfaction Score']].sort_values(by="Satisfaction Score", ascending=False).head(10)

,MSISDN/Number,Satisfaction Score
23613,33667725464,4.719602e+09
10863,33659084281,4.237100e+09
34316,33762644658,4.011219e+09
1720,33614892860,3.865531e+09
32250,33760536639,3.714400e+09
3639,33625779332,3.693751e+09
35261,33763859490,3.571675e+09
3742,33626320676,3.429428e+09
25241,33669054076,3.424131e+09
26877,33675877202,3.404278e+09


### Regression model for Satisfaction Score

In [14]:
df_feature = pd.merge(df_engagement[['MSISDN/Number', 'Bearer Id' , 'Dur. (ms)', 'Total Data']], \
                      df_experience[['MSISDN/Number', 'Total RTT', 'Total TCP retransmission', 'Total Throughput', 'encoded']], \
                      on="MSISDN/Number")
df_feature.head()

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,Total RTT,Total TCP retransmission,Total Throughput,encoded
0,33601008617,2,37104.0,1.457411e+09,91.0,9370832.0,56781.0,62
1,33601011634,2,128360.0,6.547231e+08,39.0,110232.0,42416.0,301
2,33601021217,1,38416.0,6.300924e+08,160.0,14594645.0,19256.0,54
3,33601031129,1,55730.0,6.370531e+08,60.0,2325497.0,38190.0,60
4,33601034530,2,73674.0,1.632761e+09,656.0,2006261.0,8539.0,51


In [15]:
df_feature = pd.merge(df_feature, \
                      df_satisfaction[['MSISDN/Number', 'Satisfaction Score']], \
                      on="MSISDN/Number")
df_feature.head()

,MSISDN/Number,Bearer Id,Dur. (ms),Total Data,Total RTT,Total TCP retransmission,Total Throughput,encoded,Satisfaction Score
0,33601008617,2,37104.0,1.457411e+09,91.0,9370832.0,56781.0,62,1.731452e+08
1,33601011634,2,128360.0,6.547231e+08,39.0,110232.0,42416.0,301,2.328292e+08
2,33601021217,1,38416.0,6.300924e+08,160.0,14594645.0,19256.0,54,2.483012e+08
3,33601031129,1,55730.0,6.370531e+08,60.0,2325497.0,38190.0,60,2.405565e+08
4,33601034530,2,73674.0,1.632761e+09,656.0,2006261.0,8539.0,51,2.593272e+08


In [16]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
X = df_feature.loc[:, ['Bearer Id', 'Dur. (ms)', 'Total Data', 'Total RTT', 'Total TCP retransmission', 'Total Throughput', 'encoded']].values
X = StandardScaler().fit_transform(X)
X.shape

(37274, 7)

In [17]:
y =  df_feature.loc[:, ['Satisfaction Score']].values
y = StandardScaler().fit_transform(y)
y.shape

(37274, 1)

In [18]:
reg = LinearRegression().fit(X, y)
reg.score(X, y)

0.4576047380490228

In [19]:
#save model
pickle.dump(reg, open('../models/satisfaction_model.sav', 'wb'))

## K means on the engagement and experience scores

In [20]:
from sklearn.preprocessing import StandardScaler
x_score = df_satisfaction[['engagement score', 'experience score']].loc[:, :].values
x_score = StandardScaler().fit_transform(x_score)
x_score.shape

(37274, 2)

In [21]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(x_score)
df_satisfaction['cluster'] = kmeans.labels_

In [22]:
df_satisfaction.head()

,MSISDN/Number,engagement score,experience score,Satisfaction Score,cluster
0,33601008617,3.411151e+08,5.175208e+06,1.731452e+08,0
1,33601011634,4.615728e+08,4.085460e+06,2.328292e+08,0
2,33601021217,4.862035e+08,1.039897e+07,2.483012e+08,0
3,33601031129,4.792428e+08,1.870192e+06,2.405565e+08,0
4,33601034530,5.164648e+08,2.189551e+06,2.593272e+08,0


### Average engagement  and experience scores per cluster

In [23]:
df_satisfaction[['engagement score', 'experience score', 'cluster']].groupby('cluster').mean()

,engagement score,experience score
cluster,,
0,5.556353e+08,1.700461e+07
1,6.688558e+08,2.907957e+09


In [25]:
df_satisfaction.to_csv('../data/user_experience_scores.csv', index=False)